In [45]:
using Revise
using LinearAlgebra

import PyCall; py = PyCall
import StatsBase; stat = StatsBase
import PyPlot; plt = PyPlot
import CMF; cmf = CMF
sep = CMF.Separable

include("./sep_plot_settings.jl")

set_size

## Load data

In [30]:
scipy_io = py.pyimport("scipy.io")
matfile = scipy_io.loadmat("../../cmf_data/MackeviciusData.mat")
neural = matfile["NEURAL"];

# plt.imshow(neural, aspect=5)
# plt.grid(false)
# plt.show()

In [39]:
K = 3
L = ceil(Int, 0.4 * Int(matfile["VIDEOfs"][1]))
train_test_split = floor(Int, 0.5 * size(neural, 2))

neural_train = neural[:, 1:train_test_split]
neural_test = neural[:, train_test_split+1:end];

## Fit with HALS

In [58]:
rh = CMF.fit_cnmf(
    neural_train, K=K, L=L, alg=:hals, 
    max_iter=Inf, max_time=5
);

println("Train error: ", cmf.evaluate_mse(rh))
println("Test error: ", cmf.evaluate_test(rh, neural_test))
println("Iterations: ", cmf.evaluate_convergence(rh))

Train error: 0.7374918512269535
Test error: 0.8935492200319627
Iterations: 52


## Fit with LECS

In [60]:
rs = cmf.fit_cnmf(
    neural_train, K=K, L=L, alg=:sep,
    thresh=1,
    refit_H=false,
    refit_W=false,
    verbose=false,
    spectral=false
);

rsh = CMF.fit_cnmf(
    neural_train, K=K, L=L, alg=:hals, 
    max_iter=Inf, max_time=5,
    initW=rs.W, initH=rs.H
);

println("Initial Train error: ", cmf.evaluate_mse(rs))
println("----")
println("Train error: ", cmf.evaluate_mse(rsh))
println("Test error: ", cmf.evaluate_test(rsh, neural_test))
println("Iterations: ", cmf.evaluate_convergence(rsh))

Initial Train error: 0.9055557275411709
----
Train error: 0.733019131667818
Test error: 0.8905240821002839
Iterations: 11
